In [ ]:
def oog2spec_via_cupy(audio_data):
    
    audio_data = cp.array(audio_data)
    
    # handles NaNs
    mean_signal = cp.nanmean(audio_data)
    audio_data = cp.nan_to_num(audio_data, nan=mean_signal) if cp.isnan(audio_data).mean() < 1 else cp.zeros_like(audio_data)
    
    # to spec.
    frequencies, times, spec_data = cupy_signal.spectrogram(
        audio_data, 
        fs=default_config["SAMPLE_RATE"], 
        nfft=default_config["N_FFT"], 
        nperseg=default_config["WIN_SIZE"], 
        noverlap=default_config["WIN_LAP"], 
        window='hann'
    )
    
    # Filter frequency range
    valid_freq = (frequencies >= default_config["MIN_FREQ"]) & (frequencies <= default_config["MAX_FREQ"])
    spec_data = spec_data[valid_freq, :]
    
    # Log
    spec_data = cp.log10(spec_data + 1e-20)
    
    # min/max normalize
    spec_data = spec_data - spec_data.min()
    spec_data = spec_data / spec_data.max()
    
    return spec_data.get()

In [ ]:
meta_data = pd.read_csv(f"{default_config['DATA_PATH']}/train_metadata.csv")
# meta_data = pd.read_csv(f"inputs/additional_data/BirdClef2024_additional2.csv")
meta_data.drop_duplicates(inplace=True)

meta_data.reset_index(drop=True, inplace=True)
meta_data = meta_data[["filename","primary_label"]]
meta_data

In [ ]:
f = open("data/duplicates.txt", 'r').readlines()
dup = [row.split(',')[0] for row in f]
print(f"Get {len(dup)} duplicates file")
for d in dup:
    meta_data = meta_data[meta_data.filename!=d]
meta_data.reset_index(drop=True)
meta_data.drop_duplicates(subset=['filename'], inplace=True)
meta_data.reset_index(drop=True, inplace=True)
meta_data

In [ ]:
all_bird_data = dict()

for i, row_metadata in tqdm(meta_data.iterrows()):

        try:
                audio_data, _ = librosa.load(f"inputs/train_audio/{row_metadata.primary_label}/{row_metadata.filename}.wav", sr=default_config["SAMPLE_RATE"])
        except:
                continue

        n_copy = math.ceil(5 * default_config["SAMPLE_RATE"] / len(audio_data))
        if n_copy > 1: audio_data = np.concatenate([audio_data]*n_copy)

        # for start_idx in range(0,len(audio_data)-(len(audio_data)%(5*default_config["SAMPLE_RATE"])),5*default_config["SAMPLE_RATE"]):
        # start_idx = int(start_idx)
        # start_idx = int(len(audio_data) / 2 - 2.5 * default_config["SAMPLE_RATE"])
        start_idx = 0
        end_idx = int(start_idx + 5.0 * default_config["SAMPLE_RATE"])
        input_audio = audio_data[start_idx:end_idx]

        # ogg to spec.
        input_spec = oog2spec_via_cupy(input_audio)

        input_spec = cv2.resize(input_spec, (256, 256), interpolation=cv2.INTER_AREA)

        all_bird_data[f"{row_metadata.primary_label}/{row_metadata.filename}_{start_idx}"] = input_spec.astype(np.float32)
# save to file
np.save(os.path.join("data", f'spec_first_5sec_256_256.npy'), all_bird_data)